<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/edit-carbon-configuration/ALBERT-Base-V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet-ml
!pip install cumulator
!pip install geopy
!pip install GPUtil
!pip install py-cpuinfo
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install codecarbon
!pip install transformers
!pip install tokenizers

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import csv
hardwareFile = "/usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv"
cpu_data = ['13th Gen Intel(R) Core(TM) i9-13950HX', 55]

row_present = False
with open(hardwareFile, 'r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        if row == cpu_data:
            row_present = True
            break

if not row_present:
    with open(hardwareFile, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(cpu_data)


comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')
experiment = comet_ml.Experiment(api_key="uyaIbaxDfy7jK9j7u4KNxeKNg",project_name="comet-bert-experiment",)

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 20:50:46] [setup] RAM Tracking...
[codecarbon INFO @ 20:50:46] [setup] GPU Tracking...
[codecarbon INFO @ 20:50:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:50:46] [setup] CPU Tracking...
[codecarbon WARNING @ 20:50:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 20:50:47] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13950HX
[codecarbon INFO @ 20:50:47] >>> Tracker's metadata:
[codecarbon INFO @ 20:50:47]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 20:50:47]   Python version: 3.10.12
[codecarbon INFO @ 20:50:47]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 20

In [5]:
hardwareFile = "/usr/local/lib/python3.10/dist-packages/cumulator/hardware_data/cpu.csv"
cpu_data = ['Core i9-13950HX', 'Raptor Lake-HX', '1 / 23','2.2 to 5.5 GHz','BGA 1964 ','10 nm', '36 MB', '55 W', 55]

row_present = False
with open(hardwareFile, 'r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        if row == cpu_data:
            row_present = True
            break

if not row_present:
    with open(hardwareFile, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(cpu_data)

In [6]:
baseFile = "/usr/local/lib/python3.10/dist-packages/cumulator/base.py"
with open(baseFile, 'r') as file:
        content = file.read()
with open(baseFile, 'w') as file:
        file.write(content.replace("src.cumulator", "cumulator"))
from cumulator import base
cumulator = base.Cumulator()
cumulator.on()

CPU recognized: TDP set to 55


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

raw_datasets = load_dataset("squad")
raw_datasets

[codecarbon INFO @ 20:51:02] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 20:51:02] Energy consumed for all GPUs : 0.000042 kWh. Total GPU Power : 10.131045441149675 W
[codecarbon INFO @ 20:51:02] Energy consumed for all CPUs : 0.000115 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 20:51:02] 0.000181 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:51:17] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 20:51:18] Energy consumed for all GPUs : 0.000072 kWh. Total GPU Power : 7.196330123064666 W
[codecarbon INFO @ 20:51:18] Energy consumed for all CPUs : 0.000229 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 20:51:18] 0.000350 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:51:32] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 20:51:33] Energy consumed for all GPUs : 0.000105 kWh. Total GPU Power : 7.91385086080163

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [8]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france?[SEP] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend "venite ad me omnes". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome), is a simple, modern stone statue of mary.[SEP]'

In [9]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [10]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 21:03:33] Energy consumed for RAM : 0.001230 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:03:33] Energy consumed for all GPUs : 0.001361 kWh. Total GPU Power : 7.998510293652344 W
[codecarbon INFO @ 21:03:33] Energy consumed for all CPUs : 0.005844 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 21:03:33] 0.008435 kWh of electricity used since the beginning.


(87599, 88638)

In [11]:
'''
similar to training data except
  -without start and end positions
  -simplified offset mapping
  -uses example ids
'''
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [12]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

[codecarbon INFO @ 21:03:48] Energy consumed for RAM : 0.001254 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:03:48] Energy consumed for all GPUs : 0.001383 kWh. Total GPU Power : 5.2986168532146065 W
[codecarbon INFO @ 21:03:48] Energy consumed for all CPUs : 0.005958 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 21:03:48] 0.008596 kWh of electricity used since the beginning.


(10570, 10808)

post processing

In [13]:
import evaluate
metric = evaluate.load("squad")

In [14]:
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        #experiment.set_epoch(epoch)
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

In [15]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "albert-finetuned-squad",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [17]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
[codecarbon INFO @ 21:03:53] Energy consumed for RAM : 0.001262 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:03:53] Energy consumed for all GPUs : 0.001401 kWh. Total GPU Power : 12.343146279615127 W
[codecarbon INFO @ 21:03:53] Energy consumed for all CPUs : 0.005997 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 21:03:53] 0.008660 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ------------------------------------

Epoch,Training Loss,Validation Loss
1,1.015300,No log
2,0.688500,No log
3,0.476800,No log


[codecarbon INFO @ 21:04:11] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:04:11] Energy consumed for all GPUs : 0.000427 kWh. Total GPU Power : 102.52177751190213 W
[codecarbon INFO @ 21:04:11] Energy consumed for all CPUs : 0.000115 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 21:04:11] 0.000566 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:04:26] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:04:26] Energy consumed for all GPUs : 0.000886 kWh. Total GPU Power : 110.13626899546355 W
[codecarbon INFO @ 21:04:26] Energy consumed for all CPUs : 0.000229 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 21:04:26] 0.001163 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:04:41] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 21:04:41] Energy consumed for all GPUs : 0.001369 kWh. Total GPU Power : 115.95959261915

TrainOutput(global_step=33240, training_loss=0.7268672938524744, metrics={'train_runtime': 3400.7888, 'train_samples_per_second': 78.192, 'train_steps_per_second': 9.774, 'total_flos': 4404294320329728.0, 'train_loss': 0.7268672938524744, 'epoch': 3.0})

In [18]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 82.80983916745507, 'f1': 90.01321161949743}

In [19]:
cumulator.off()
cumulator.on()

In [20]:
cumulator.display_carbon_footprint()

########
Overall carbon footprint: 1.10e+01 gCO2eq
########
Carbon footprint due to computations: 1.10e+01 gCO2eq
Carbon footprint due to communications: 0.00e+00 gCO2eq
This carbon footprint is equivalent to 3.12e-04 square meters of u.s. forests preserved from conversion to cropland in one year.


In [21]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/noushsuon/albert-finetuned-squad/commit/288e280399438eba5784502439eb81dd5f7d8641', commit_message='Training complete', commit_description='', oid='288e280399438eba5784502439eb81dd5f7d8641', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [23]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [24]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [25]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [26]:
from huggingface_hub import Repository, get_full_repo_name
model_name = "albert-finetuned-squad"
repo_name = get_full_repo_name(model_name)
repo_name

'noushsuon/albert-finetuned-squad'

In [27]:
from huggingface_hub import Repository
output_dir = "albert-finetuned-squad-accelerate" + model_checkpoint

repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/noushsuon/albert-finetuned-squad into local empty directory.


Download file model.safetensors:   0%|          | 15.6k/42.3M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 4.43k/4.43k [00:00<?, ?B/s]

Clean file training_args.bin:  23%|##2       | 1.00k/4.43k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/42.3M [00:00<?, ?B/s]

In [28]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/33240 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1351 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 80.93661305581836, 'f1': 89.04875585617638}
Evaluation!


...

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 1: {'exact_match': 80.49195837275307, 'f1': 88.75302948208136}
Evaluation!


  0%|          | 0/1351 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 80.0473036896878, 'f1': 88.62879550502866}


In [29]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [30]:
experiment.end()

In [31]:
cumulator.off()
cumulator.display_carbon_footprint()

########
Overall carbon footprint: 1.95e+01 gCO2eq
########
Carbon footprint due to computations: 1.95e+01 gCO2eq
Carbon footprint due to communications: 0.00e+00 gCO2eq
This carbon footprint is equivalent to 7.23e-03 liters of diesel consumed.
